In [1]:
import pandas as pd

In [2]:
#There are 9 million rows in the original dataset.

#I was only able to obtain up to a million rows, but that's a sufficient amount of data

In [3]:
#Feed and Read the Data

data = pd.read_csv("nyc-property-data")

In [4]:
data

,bble,boro,block,lot,owner,bldgcl,taxclass,ltfront,ltdepth,stories,...,bin,nta,geocoded_column,:@computed_region_efsh_h5xi,:@computed_region_f5dn_yrer,:@computed_region_yeji_bk3q,:@computed_region_92fq_4b7q,:@computed_region_sbqj_enih,ext,excd2
0,1000163859,1,16,3859,"CHEN, QI TOM",R4,2,0,0,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000730028,1,73,28,NYC DSBS,V1,4,183,52,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000730029,1,73,29,NYC DSBS,Y7,4,90,500,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000297504,1,29,7504,NaN,R0,2,36,73,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000360012,1,36,12,NYC DSBS,Y7,4,534,604,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,5001430028,5,143,28,DILBERIAN SERVICE MEM,A1,1,45,145,2.0,...,5170594.0,New Brighton-Silver Lake,"{'type': 'Point', 'coordinates': [-74.106353, ...",10697.0,4.0,1.0,13.0,74.0,NaN,NaN
999996,4161691001,4,16169,1001,"KAISER, ANGELIKA",R4,2,0,0,11.0,...,4539425.0,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,"{'type': 'Point', 'coordinates': [-73.81704, 4...",20531.0,51.0,3.0,41.0,58.0,NaN,NaN
999997,4162740006,4,16274,6,"LYNCH, ANDREA H",B1,1,100,80,1.6,...,4306107.0,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,"{'type': 'Point', 'coordinates': [-73.858287, ...",20532.0,51.0,3.0,41.0,58.0,G,NaN
999998,4162130044,4,16213,44,SANFORD BERNSTEIN,B2,1,42,100,2.5,...,4304487.0,Breezy Point-Belle Harbor-Rockaway Park-Broad ...,"{'type': 'Point', 'coordinates': [-73.839201, ...",20532.0,51.0,3.0,41.0,58.0,EG,NaN


In [5]:
#The latitude & longitude data is very important because it gives us the geo-positions for the properties
#that we could use for plotting

len(data[["latitude", "longitude"]].dropna())

947373

In [6]:
#We'll get the important metrics we need to do our analysis

#Key "bble" = Boro, Block, Lot and Easement code
#av_land = Actual Land Value
#av_tot = Actual Total Value
#As well as latitude & longitude

keys = ["bble", "avland", "avtot", "latitude", "longitude"]

In [7]:
data = data[keys]
data

,bble,avland,avtot,latitude,longitude
0,1000163859,3310,159381,NaN,NaN
1,1000730028,1581750,1581750,NaN,NaN
2,1000730029,2812050,3696750,NaN,NaN
3,1000297504,0,0,NaN,NaN
4,1000360012,9180000,11810700,NaN,NaN
...,...,...,...,...,...
999995,5001430028,14180,33393,40.633763,-74.106353
999996,4161691001,9636,90107,40.583401,-73.817040
999997,4162740006,15161,63957,40.574714,-73.858287
999998,4162130044,6182,26153,40.579731,-73.839201


In [8]:
#Set index to the unique "file key" for the buildings
data.set_index("bble", inplace=True)


#Convert types to suitable numeric values
data = data.apply(pd.to_numeric)

In [9]:
#Getting The Location Slice:

#The location slice helps us to find the upper and lower bounds for both latitude and longitude.

def location_slice():
    map_points = data[["latitude", "longitude"]]
    
    lat_max = map_points.max()["latitude"]
    lat_min = map_points.min()["latitude"]
    lon_max = map_points.max()["longitude"]
    lon_min = map_points.min()["longitude"]
    
    print("The max latitude is", lat_max)
    print("The min latitude is", lat_min)
    print("The max longitude is {}".format(lon_max))
    print("The min longitude is {}".format(lon_min))

In [10]:
location_slice()

The max latitude is 40.912883
The min latitude is 40.540776
The max longitude is -73.700376
The min longitude is -74.187856


In [11]:
#Now that we know the upper & lower bounds, we can find the distance from bound to bound. This will be what we slice up for
#our grid. We'll modify the function, and get the variables as such below.

def locationSlice(slice_constant):
    map_points = data[["latitude", "longitude"]]
    
    lat_max = map_points.max()["latitude"]
    lat_min = map_points.min()["latitude"]
    lon_max = map_points.max()["longitude"]
    lon_min = map_points.min()["longitude"]

    lat_range = (lat_max - lat_min)/slice_constant
    lon_range = (lon_max - lon_min)/ slice_constant
    
    return (lat_max, lat_min, lon_max, lon_min, lat_range, lon_range)

In [12]:
lat_max, lat_min, lon_max, lon_min, lat_range, lon_range = locationSlice(100)

In [13]:
#Building The Grid:

#We have (x-minNum)/(maxNum-minNum) to return the percentile, and then multiply it by the slice constant because that is how
#many groups we are creating. We use int() function to get rid of decimal. Finally, we need the if statement because at the 
#maximum we would get what our slice constant equals, but we really want 0 to slice constant-1.

def LocationGroup(x, maxNum, minNum, slice_constant):
    if x == maxNum:
        loc = int(slice_constant * (x-minNum)/ (maxNum-minNum))-1
    else:
        loc = int(slice_constant * (x-minNum)/ (maxNum-minNum))
    return loc

In [14]:
#Check for null values in the data
#The function below returns a truth series on if a column is null or not

print(data["latitude"].notnull())

bble
1000163859    False
1000730028    False
1000730029    False
1000297504    False
1000360012    False
              ...  
5001430028     True
4161691001     True
4162740006     True
4162130044     True
5001101056     True
Name: latitude, Length: 1000000, dtype: bool


In [15]:
print(data[data["latitude"].notnull()])

            avland  avtot   latitude  longitude
bble                                           
1009287504       0      0  40.737730 -73.980554
1010607502       0      0  40.763878 -73.989853
1004207501       0      0  40.719649 -73.991800
1004237501       0      0  40.718795 -73.994657
1004347504       0      0  40.725912 -73.983649
...            ...    ...        ...        ...
5001430028   14180  33393  40.633763 -74.106353
4161691001    9636  90107  40.583401 -73.817040
4162740006   15161  63957  40.574714 -73.858287
4162130044    6182  26153  40.579731 -73.839201
5001101056    5673   5673  40.634501 -74.086877

[947373 rows x 4 columns]


In [16]:
#Modify our data to be without the null values

data = data[data["latitude"].notnull()]

In [17]:
#We can make use of the function dataframe.apply() to apply the functions we created to the pandas dataframe/ columns. 
#We also create a lambda function to have the minimum, maximum and slice constant plugged in when we apply.

print(data["latitude"].apply(lambda x: LocationGroup(x, lat_max, lat_min, 100)))

bble
1009287504    52
1010607502    59
1004207501    48
1004237501    47
1004347504    49
              ..
5001430028    24
4161691001    11
4162740006     9
4162130044    10
5001101056    25
Name: latitude, Length: 947373, dtype: int64


In [18]:
#We'll then apply/ create the groups of latitude and longitude

data["Lat Group"] = data["latitude"].apply((lambda x: LocationGroup(x, lat_max, lat_min, 100)))

data["Lon Group"] = data["longitude"].apply((lambda x: LocationGroup(x, lon_max, lon_min, 100)))

In [19]:
data

,avland,avtot,latitude,longitude,Lat Group,Lon Group
bble,,,,,,
1009287504,0,0,40.737730,-73.980554,52,42
1010607502,0,0,40.763878,-73.989853,59,40
1004207501,0,0,40.719649,-73.991800,48,40
1004237501,0,0,40.718795,-73.994657,47,39
1004347504,0,0,40.725912,-73.983649,49,41
...,...,...,...,...,...,...
5001430028,14180,33393,40.633763,-74.106353,24,16
4161691001,9636,90107,40.583401,-73.817040,11,76
4162740006,15161,63957,40.574714,-73.858287,9,67


In [20]:
#Next, we'll throw out anything that has 0 for avtot (Actual Total Value)
data = data[data["avtot"] > 0]

In [21]:
#Location Codes:

#Next, we want a way to represent both the latitude and longitude together.
#This can be done as follows:

print(data["Lat Group"].astype(str) + "-" + data["Lon Group"].astype(str))

bble
2022961001    71-53
2027070009    73-59
2052320055    90-72
3022041003    43-46
3022301304    42-47
              ...  
5001430028    24-16
4161691001    11-76
4162740006     9-67
4162130044    10-71
5001101056    25-20
Length: 937527, dtype: object


In [22]:
#This creates our location codes for the properties

loc_code = data["Lat Group"].astype(str) + "-" + data["Lon Group"].astype(str)

data.loc[:, "Loc Code"] = loc_code

C:\Users\victo\AppData\Local\Temp\ipykernel_24016\906937428.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, "Loc Code"] = loc_code


In [23]:
data

,avland,avtot,latitude,longitude,Lat Group,Lon Group,Loc Code
bble,,,,,,,
2022961001,3975,84868,40.806519,-73.926697,71,53,71-53
2027070009,4425,15801,40.815468,-73.899614,73,59,73-59
2052320055,7761,7761,40.878905,-73.834257,90,72,90-72
3022041003,1196,39603,40.701925,-73.960234,43,46,43-46
3022301304,12785,25571,40.699780,-73.957567,42,47,42-47
...,...,...,...,...,...,...,...
5001430028,14180,33393,40.633763,-74.106353,24,16,24-16
4161691001,9636,90107,40.583401,-73.817040,11,76,11-76
4162740006,15161,63957,40.574714,-73.858287,9,67,9-67


In [24]:
#We can turn our data into a dataset of each location code, and their average/ mean values for each variable.

data.groupby("Loc Code").mean()

,avland,avtot,latitude,longitude,Lat Group,Lon Group
Loc Code,,,,,,
0-4,2897.000000,4.559500e+04,40.540776,-74.167351,0.0,4.0
10-35,16665.000000,2.953950e+04,40.578401,-74.012627,10.0,35.0
10-36,122234.788462,1.593950e+05,40.579268,-74.009564,10.0,36.0
10-37,11747.507163,4.050412e+04,40.579548,-74.004742,10.0,37.0
10-38,303539.200000,1.571724e+06,40.579190,-74.002064,10.0,38.0
...,...,...,...,...,...,...
99-57,822328.846154,5.515507e+06,40.910706,-73.906369,99.0,57.0
99-58,83039.845304,2.668646e+05,40.910688,-73.901827,99.0,58.0
99-59,18545.148387,9.142069e+04,40.910545,-73.898564,99.0,59.0


In [25]:
data = data.groupby("Loc Code").mean()

In [26]:
#We return to the previous values from above (i.e the upper & lower bounds of the latitude and longitude)
#We also create the increments

slice_constant = 100

latIncrement = (lat_max - lat_min)/ slice_constant
lonIncrement = (lon_max - lon_min)/ slice_constant

print([lat_max,lat_min, lon_max, lon_min])

print("")
print("Lat Increment:", latIncrement)
print("Lon Increment:", lonIncrement)

[40.912883, 40.540776, -73.700376, -74.187856]

Lat Increment: 0.0037210699999999973
Lon Increment: 0.004874799999999908


In [27]:
#We are going to give each location code a lower, middle, and upper bound. The way we will do it is by multiplying by the 
#lat or lon group.

data["Lat 0"] = lat_min + data["Lat Group"] * latIncrement
data["Lat 1"] = lat_min + (data["Lat Group"]+ 0.5) * latIncrement
data["Lat 2"] = lat_min + (data["Lat Group"] + 1) * latIncrement

data["Lon 0"] = lon_min + data["Lon Group"] * lonIncrement
data["Lon 1"] = lon_min + (data["Lon Group"] + 0.5) * lonIncrement
data["Lon 2"] = lon_min + (data["Lon Group"] + 1) * lonIncrement

In [28]:
data

,avland,avtot,latitude,longitude,Lat Group,Lon Group,Lat 0,Lat 1,Lat 2,Lon 0,Lon 1,Lon 2
Loc Code,,,,,,,,,,,,
0-4,2897.000000,4.559500e+04,40.540776,-74.167351,0.0,4.0,40.540776,40.542637,40.544497,-74.168357,-74.165919,-74.163482
10-35,16665.000000,2.953950e+04,40.578401,-74.012627,10.0,35.0,40.577987,40.579847,40.581708,-74.017238,-74.014801,-74.012363
10-36,122234.788462,1.593950e+05,40.579268,-74.009564,10.0,36.0,40.577987,40.579847,40.581708,-74.012363,-74.009926,-74.007488
10-37,11747.507163,4.050412e+04,40.579548,-74.004742,10.0,37.0,40.577987,40.579847,40.581708,-74.007488,-74.005051,-74.002614
10-38,303539.200000,1.571724e+06,40.579190,-74.002064,10.0,38.0,40.577987,40.579847,40.581708,-74.002614,-74.000176,-73.997739
...,...,...,...,...,...,...,...,...,...,...,...,...
99-57,822328.846154,5.515507e+06,40.910706,-73.906369,99.0,57.0,40.909162,40.911022,40.912883,-73.909992,-73.907555,-73.905118
99-58,83039.845304,2.668646e+05,40.910688,-73.901827,99.0,58.0,40.909162,40.911022,40.912883,-73.905118,-73.902680,-73.900243
99-59,18545.148387,9.142069e+04,40.910545,-73.898564,99.0,59.0,40.909162,40.911022,40.912883,-73.900243,-73.897805,-73.895368


In [29]:
#Save the data above
data.to_csv("NYC's-Map-Data.csv", encoding="UTF-8")

In [30]:
#Creating And Plotting The Properties On The Map:

#First we import gmplot (if you've already installed it) 
#Else install gmplot by passing the code: "!pip install gmplot"

#In order to use gmplot, you need to get an apikey from Google (which you can get for free)

#It is best practice to encode the apikey in a private/ local file and access it as needed

In [31]:
import gmplot

In [32]:
def read_api_key(file):
    """
    Read the Api Key from the file.
    
    Args:
        file (string): File containing API Key
        
    Returns:
        The Api Key and can be assigned to a variable
    """
    
    with open(file, 'r') as f:
        return f.read()

In [33]:
apikey = read_api_key("api_key.txt")

In [34]:
#Next we create the base map
#You can access the map locally in your working directory

def create_Map():
    gmap = gmplot.GoogleMapPlotter.from_geocode("New York City", apikey=apikey)
    gmap.draw("NYC Real Estate.html")
    

create_Map()

In [35]:
print([lat_max,lat_min, lon_max, lon_min])

[40.912883, 40.540776, -73.700376, -74.187856]


In [36]:
#To draw a square, we are going to need 4 points. We feed these points into the map as an array of x and y-coordinates

#The code below creates a square around the grid (i.e from South Yonkers to Staten Island)

latMax = 40.912883  #40.896572
latMin = 40.540776
lonMax = -73.718032
lonMin = -74.187856


def Create_Map():
    gmap = gmplot.GoogleMapPlotter.from_geocode("New York City", apikey=apikey)
    xPoints = [latMax, latMin, latMin, latMax]
    yPoints = [lonMin, lonMin, lonMax, lonMax]
    
    gmap.polygon(xPoints, yPoints, color = "#ff0000")
    gmap.draw("NYC Real Estate.html")

Create_Map()

In [37]:
#Read the data to use to plot on the map

MapData = pd.read_csv("NYC's-Map-Data.csv", encoding="UTF-8", index_col="Loc Code")

In [38]:
MapData.rename(columns={"avtot": "Total Value"}, inplace=True)

In [39]:
MapData

,avland,Total Value,latitude,longitude,Lat Group,Lon Group,Lat 0,Lat 1,Lat 2,Lon 0,Lon 1,Lon 2
Loc Code,,,,,,,,,,,,
0-4,2897.000000,4.559500e+04,40.540776,-74.167351,0.0,4.0,40.540776,40.542637,40.544497,-74.168357,-74.165919,-74.163482
10-35,16665.000000,2.953950e+04,40.578401,-74.012627,10.0,35.0,40.577987,40.579847,40.581708,-74.017238,-74.014801,-74.012363
10-36,122234.788462,1.593950e+05,40.579268,-74.009564,10.0,36.0,40.577987,40.579847,40.581708,-74.012363,-74.009926,-74.007488
10-37,11747.507163,4.050412e+04,40.579548,-74.004742,10.0,37.0,40.577987,40.579847,40.581708,-74.007488,-74.005051,-74.002614
10-38,303539.200000,1.571724e+06,40.579190,-74.002064,10.0,38.0,40.577987,40.579847,40.581708,-74.002614,-74.000176,-73.997739
...,...,...,...,...,...,...,...,...,...,...,...,...
99-57,822328.846154,5.515507e+06,40.910706,-73.906369,99.0,57.0,40.909162,40.911022,40.912883,-73.909992,-73.907555,-73.905118
99-58,83039.845304,2.668646e+05,40.910688,-73.901827,99.0,58.0,40.909162,40.911022,40.912883,-73.905118,-73.902680,-73.900243
99-59,18545.148387,9.142069e+04,40.910545,-73.898564,99.0,59.0,40.909162,40.911022,40.912883,-73.900243,-73.897805,-73.895368


In [40]:
#We can plot each individual square of the grid based on the nyc data

def Create_Map():
    place = "New York City"
    gmap = gmplot.GoogleMapPlotter.from_geocode(place, apikey=apikey)
    
    for index, row in MapData.iterrows():
        xPoints = [row["Lat 2"], row["Lat 0"], row["Lat 0"], row["Lat 2"]]
        yPoints = [row["Lon 0"], row["Lon 0"], row["Lon 2"], row["Lon 2"]]
        
        gmap.polygon(xPoints, yPoints, color="#ff0000")
        
    gmap.draw("NYC Real Estate.html")

Create_Map()

In [41]:
#Normalize the rank to fit between the range 0 - 1 using pct = True
#The percentile rank will help us to rank each location code, thus we could either give the color red for locations
#with a low average value or green for locations with a high average value


MapData["Rank"] = MapData["Total Value"].rank(pct=True)
MapData["Rank"]

Loc Code
0-4      0.189150
10-35    0.071848
10-36    0.548974
10-37    0.148094
10-38    0.895601
           ...   
99-57    0.961584
99-58    0.658358
99-59    0.395894
99-68    0.869795
99-69    0.775073
Name: Rank, Length: 3410, dtype: float64

In [42]:
#Finishing The Map

#The finished map should have red squares if the value of the property is low,
#Grey/ Mix of red and green for average property values
#And green squares, if the property value is high

def Create_Map():
    gmap = gmplot.GoogleMapPlotter.from_geocode("New York City", apikey=apikey)
    
    for index, row in MapData.iterrows():
        xPoints = [row["Lat 2"], row["Lat 0"], row["Lat 0"], row["Lat 2"]]
        yPoints = [row["Lon 0"], row["Lon 0"], row["Lon 2"], row["Lon 2"]]
        
        if row["Rank"]<=0.5:
            color ='#%02X%02X%02X' % (int(255*((1-row["Rank"]*2))),0,0)
        else:
            color ='#%02X%02X%02X' % (0,int(255*((row["Rank"]-.5)*2)),0)
        
        gmap.polygon(xPoints, yPoints, color=color)
        
    gmap.draw("NYC Real Estate.html")

Create_Map()